# Heuristics for Link Prediction on Graphs - Methods

In [1]:
import networkx as nx
import random
import folium
import math
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score
import numpy as np


def set_up_random_seed(seed):
    random.seed(seed)
    return seed

def katz_index(adj_matrix, beta=0.1, max_l=10):
    """
    Compute Katz index.

    adj_matrix : np.ndarray
        The adjacency matrix of the graph.
    beta : float
        The decaying factor.
    max_l : int
        Maximum length of walks.

    Returns : np.ndarray
        The Katz index matrix.
    """
    n = adj_matrix.shape[0]
    katz = np.zeros((n, n))
    for l in range(1, max_l + 1):
        katz += beta**l * np.linalg.matrix_power(adj_matrix, l)
    return katz

def rooted_pagerank(adj_matrix, alpha=0.85):
    """
    Compute Rooted PageRank.

    adj_matrix : np.ndarray
        The adjacency matrix of the graph.
    alpha : float
        Jump probability.

    Returns : np.ndarray
        The RPR matrix.
    """
    n = adj_matrix.shape[0]
    p = np.eye(n)
    r = (1-alpha) * np.linalg.inv(np.eye(n) - alpha * adj_matrix)
    return r @ p


def random_link_prediction(G, num_predictions):
    """
    Generate random link predictions for the graph G.

    :param G: NetworkX graph
    :param num_predictions: Number of link predictions to generate
    :return: Set of randomly predicted links
    """
    non_edges = list(nx.non_edges(G))
    random_predictions = random.sample(non_edges, num_predictions) if non_edges else []
    return set(random_predictions)

def visualize_graph(graph1, graph2, title1, title2, removed_edges,top_k_limks ):
    pos = nx.spring_layout(graph1, seed=42)  # Define the layout for our graph visualization

    # Original Graph
    plt.figure(figsize=(8, 6))  # Set the figure size
    nx.draw(graph1, pos, with_labels=True, node_size=700, node_color='skyblue', font_size=15, edge_color='gray')
    plt.title(title1)  # Set the title for our graph
    plt.show()

    # Links Removed
    # plt.figure(figsize=(8, 6))
    # nx.draw(graph2, pos, with_labels=True, node_size=700, node_color='skyblue', font_size=15, edge_color='gray')
    # plt.title(title2)  # Set the title for our graph
    # plt.show()

    # Visualising links removed
    # Draw nodes
    plt.figure(figsize=(8, 6))
    nx.draw_networkx_nodes(graph2, pos, node_size=700, node_color='skyblue')

    # If removed edges are provided, visualize them in red
    if top_k_limks:
        nx.draw_networkx_edges(graph1, pos, edgelist=top_k_limks, edge_color='blue', width=1.5)

    # Draw the remaining edges
    remaining_edges = [e for e in graph2.edges() if e not in removed_edges]
    nx.draw_networkx_edges(graph2, pos, edgelist=remaining_edges, edge_color='gray')

    nx.draw_networkx_labels(graph2, pos, font_size=15)
    plt.title("Visualising links removed")
    plt.show()

    # Visualising links predicted
    # Draw nodes
    plt.figure(figsize=(8, 6))
    nx.draw_networkx_nodes(graph2, pos, node_size=700, node_color='skyblue')

    # If removed edges are provided, visualize them in red
    if removed_edges:
        nx.draw_networkx_edges(graph2, pos, edgelist=removed_edges, edge_color='red', width=1.5)

    # Draw the remaining edges
    remaining_edges = [e for e in graph2.edges() if e not in removed_edges]
    nx.draw_networkx_edges(graph2, pos, edgelist=remaining_edges, edge_color='gray')

    nx.draw_networkx_labels(graph2, pos, font_size=15)
    plt.title("Visualising links removed")
    plt.show()


    #  Visualising links removed and predicted
    plt.figure(figsize=(8, 6))
    nx.draw_networkx_nodes(graph2, pos, node_size=700, node_color='skyblue')

    # If removed edges are provided, visualize them in red
    if removed_edges:
        nx.draw_networkx_edges(graph2, pos, edgelist=removed_edges, edge_color='red', width=2.5)
    if top_k_limks:
        nx.draw_networkx_edges(graph2, pos, edgelist=top_k_limks, edge_color='blue', width=1.5)

    # Draw the remaining edges
    remaining_edges = [e for e in graph2.edges() if e not in removed_edges]
    nx.draw_networkx_edges(graph2, pos, edgelist=remaining_edges, edge_color='gray')

    nx.draw_networkx_labels(graph2, pos, font_size=15)
    plt.title("Visualising links predicted and links removed")
    plt.show()


    return

def predict_links(G, method, x):
    # Convert x from percentage to fraction
    x = x/100.0

    # Create a copy of the graph
    G_copy = G.copy()

    # Get all the edges from the graph
    all_edges = list(G.edges())

    # Calculate the number of links to be removed
    num_removed = int(x * len(all_edges))
    # print(num_removed)

    # # Remove x percent of the links
    # removed_edges = random.sample(all_edges, num_removed)
    # G.remove_edges_from(removed_edges)

    # Get all the edges from the graph
    all_edges = list(G_copy.edges())

    # Remove x percent of the links
    removed_edges = random.sample(all_edges, num_removed)
    G_copy.remove_edges_from(removed_edges)


    # Calculate link prediction scores
    if method.lower() == 'jc':
        preds = nx.jaccard_coefficient(G_copy)
    elif method.lower() == 'aa':
        preds = nx.adamic_adar_index(G_copy)
    elif method.lower() == 'pa':
        preds = nx.preferential_attachment(G_copy)
    elif method.lower() == 'ra':  # 'ra' for Resource Allocation Index
        preds = nx.resource_allocation_index(G_copy)
    elif method.lower() == 'cn':  # 'cn' for Common Neighbors
        # This method predicts links between node pairs having a higher number of common neighbors.
        # Identify non-edges in the graph
        non_edges = list(nx.non_edges(G_copy))
        # Initialize a list to hold the predictions
        common_neighbors_scores = []
        # Calculate scores based on the number of common neighbors
        for u, v in non_edges:
            # Compute the number of common neighbors
            common_neighbors = len(list(nx.common_neighbors(G_copy, u, v)))
            # The score is the number of common neighbors. The assumption is that
            # the more common neighbors two nodes have, the more likely they are to be connected.
            score = common_neighbors
            # Append the result to the list of predictions
            common_neighbors_scores.append((u, v, score))
        # The predictions are stored in 'preds'
        preds = common_neighbors_scores
    elif method.lower() == 'tc':  # 'tc' for Triadic Closure
        # Step 1: Identify pairs of nodes with common neighbors (potential triads)
        non_edges = list(nx.non_edges(G_copy))
        triadic_closure_scores = []
        # Step 2: Assign scores based on the number of common neighbors or another suitable metric
        for u, v in non_edges:
            common_neighbors = len(list(nx.common_neighbors(G_copy, u, v)))
            # The score could be simply the number of common neighbors.
            # You might want to use a more complex metric here, depending on your specific case.
            score = common_neighbors
            triadic_closure_scores.append((u, v, score))
        preds = triadic_closure_scores
    elif method.lower() == 'random':  # Added condition for random method
        # Get the random link predictions as a list of tuples
        preds = random_link_prediction(G_copy, num_removed)
        # Convert to the required format [(u, v, score), ...] where score is dummy (e.g., 1) as it's not applicable here.
        preds = [(u, v, 1) for (u, v) in preds]  # Adding a dummy score
    elif method.lower() == 'katz':
        G_copy = nx.convert_node_labels_to_integers(G_copy, first_label=0)
        # Convert the graph to an adjacency matrix
        adj_matrix = nx.to_numpy_array(G_copy)
        scores_matrix = katz_index(adj_matrix)  # Using the custom implementation
        # Now, you must adapt this matrix of scores to the format of your predictions.
        non_edges = list(nx.non_edges(G_copy))
        preds = [(u, v, scores_matrix[u, v]) for u, v in non_edges]
    elif method.lower() == 'rpr':  # 'rpr' for Rooted PageRank
        G_copy = nx.convert_node_labels_to_integers(G_copy, first_label=0)
        # Convert the graph to an adjacency matrix
        adj_matrix = nx.to_numpy_array(G_copy)
        scores_matrix = rooted_pagerank(adj_matrix)  # Using the custom implementation
        # Similar adaptation required here as well.
        non_edges = list(nx.non_edges(G_copy))
        preds = [(u, v, scores_matrix[u, v]) for u, v in non_edges]
    else:
        raise ValueError("Invalid method!")

    # Extract top k predicted links
    k = num_removed
    # print("Number of Links Removed (and Number of Links Predicted)")
    # print(k)
    print(f"Number of Links Removed (and Number of Links Predicted): {k}")
    sorted_preds = sorted(preds, key=lambda x: x[2], reverse=True)
    top_k_links = [(u, v) for u, v, _ in sorted_preds[:k]]

    # Compute intersection of predicted links and ground truth links
    common_links = set(top_k_links).intersection(set(removed_edges))

    # Calculate accuracy
    accuracy = len(common_links) / float(k) * 100

    # Visualise both graphs
    # visualize_graph(G, G_copy, "Original Graph G","Copied Graph G_copy", removed_edges, top_k_links)

    return accuracy

def execute_link_prediction_methods(G, removal_percentage):
    """
    Execute various link prediction methods and print their accuracies.

    Parameters:
    G (networkx.Graph): The graph on which link prediction needs to be performed.
    removal_percentage (int): The percentage of links to be removed from the graph for prediction testing.
    """
    methods = [
        ('Jaccard Coefficient', 'jc'),
        ('Adamic Adar Index', 'aa'),
        ('Preferential Attachment', 'pa'),
        ('Resource Allocation Index', 'ra'),
        ('Common Neighbors', 'cn'),
        ('Triadic Closure', 'tc'),
        ('Random', 'random'),
        ('Katz Index', 'katz'),
        ('Rooted PageRank', 'rpr')
    ]

    for method_name, method_code in methods:
        print(f"Method: {method_name}")
        try:
            accuracy = predict_links(G, method_code, removal_percentage)
            print(f"Accuracy: {accuracy:.2f}%")
        except Exception as e:
            print(f"An error occurred while executing the {method_name} method: {str(e)}")
        print()

# Init Graphs for COST, BT and CORONET

In [7]:
def create_cost266_graph():
    # Create an empty graph
    G = nx.Graph()

    # Nodes
    nodes = [i for i in range(1, 38)]
    G.add_nodes_from(nodes)
    # edges computed for fiber length from haversine distance
    edges = [
            (1, 8, 259.84965666159),
            (1, 14, 1067.1610071760647),
            (1, 15, 551.932850462205),
            (1, 19, 540.2969276038555),
            (2, 26, 1362.6064844523005),
            (2, 31, 793.8895623997772),
            (2, 36, 1500),
            (3, 21, 760.3967118138279),
            (3, 22, 508.18611493655453),
            (3, 30, 1244.055742188373),
            (4, 9, 474.5629290263091),
            (4, 31, 486.21646810796915),
            (4, 36, 551.0721619498521),
            (5, 10, 539.8567721623151),
            (5, 15, 379.99508008152793),
            (5, 24, 757.6386535506992),
            (5, 28, 420.89799110496745),
            (5, 35, 774.6461442087643),
            (6, 14, 609.3320278949963),
            (6, 19, 238.79773630910168),
            (7, 21, 833.7142844783104),
            (7, 22, 757.0555809713325),
            (7, 27, 747.5017116213392),
            (8, 12, 263.47457585633975),
            (8, 27, 392.4792710554075),
            (9, 17, 435.9301886415584),
            (9, 28, 667.8119982971642),
            (10, 25, 720.4908600074742),
            (10, 32, 776.2740263923121),
            (11, 14, 462.5816015725619),
            (11, 19, 689.4418556744857),
            (12, 13, 274.66261609360544),
            (13, 15, 591.9943959052813),
            (13, 24, 456.2237094139668),
            (13, 33, 271.7320304315402),
            (16, 25, 1182.486309099843),
            (16, 32, 597.7929501950997),
            (16, 35, 1370.7464471274227),
            (17, 35, 383.0241099250181),
            (18, 19, 1977.1502858443332),
            (18, 21, 750.3016254076838),
            (18, 30, 470.97351082501496),
            (19, 27, 513.4593301745787),
            (20, 22, 410.3594835737697),
            (20, 27, 595.1118221794119),
            (20, 37, 507.63930413732726),
            (22, 29, 906.6723231378437),
            (23, 24, 521.4075026201809),
            (23, 29, 720.9026371475536),
            (23, 37, 326.44767652831126),
            (24, 34, 534.012623777059),
            (26, 29, 636.4633146922802),
            (27, 33, 600.37691157053),
            (28, 34, 375.52200677111244),
            (29, 36, 782.9414880645511),
            (33, 37, 218.2728433959167),
            (34, 36, 400.6139057896578)
            ]

    # Add Edges
    # G.add_edges_from(edges)
    G.add_weighted_edges_from(edges)

    # Node attributes (latitude and longitude)
    node_attributes = {
                        1: {"lat": 52.35, "long": 4.90},
                        2: {"lat": 38.00, "long": 23.73},
                        3: {"lat": 41.37, "long": 2.18},
                        4: {"lat": 44.83, "long": 20.50},
                        5: {"lat": 52.52, "long": 13.40},
                        6: {"lat": 52.47, "long": -1.88},
                        7: {"lat": 44.85, "long": -0.57},
                        8: {"lat": 50.83, "long": 4.35},
                        9: {"lat": 47.50, "long": 19.08},
                        10: {"lat": 55.72, "long": 12.57},
                        11: {"lat": 53.33, "long": -6.25},
                        12: {"lat": 51.23, "long": 6.78},
                        13: {"lat": 50.10, "long": 8.67},
                        14: {"lat": 55.85, "long": -4.25},
                        15: {"lat": 53.55, "long": 10.02},
                        16: {"lat": 60.17, "long": 24.97},
                        17: {"lat": 50.05, "long": 19.95},
                        18: {"lat": 38.73, "long": -9.13},
                        19: {"lat": 51.50, "long": -0.17},
                        20: {"lat": 45.73, "long": 4.83},
                        21: {"lat": 40.42, "long": -3.72},
                        22: {"lat": 43.30, "long": 5.37},
                        23: {"lat": 45.47, "long": 9.17},
                        24: {"lat": 48.13, "long": 11.57},
                        25: {"lat": 59.93, "long": 10.75},
                        26: {"lat": 38.12, "long": 13.35},
                        27: {"lat": 48.87, "long": 2.33},
                        28: {"lat": 50.08, "long": 14.43},
                        29: {"lat": 41.88, "long": 12.50},
                        30: {"lat": 37.38, "long": -5.98},
                        31: {"lat": 42.75, "long": 23.33},
                        32: {"lat": 59.33, "long": 18.05},
                        33: {"lat": 48.58, "long": 7.77},
                        34: {"lat": 48.22, "long": 16.37},
                        35: {"lat": 52.25, "long": 21.00},
                        36: {"lat": 45.83, "long": 16.02},
                        37: {"lat": 47.38, "long": 8.55}
                      }

    # Setting node attributes
    nx.set_node_attributes(G, node_attributes)

    return G


def haversine_distance(lat1, lon1, lat2, lon2):
    # Convert latitude and longitude from degrees to radians
    lat1 = math.radians(lat1)
    lon1 = math.radians(lon1)
    lat2 = math.radians(lat2)
    lon2 = math.radians(lon2)

    # Haversine formula
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = math.sin(dlat/2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon/2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    R = 6371  # Radius of the Earth in kilometers
    distance = R * c

    return distance

def calculate_edge_attributes(G):
    edge_betweenness = nx.edge_betweenness_centrality(G, normalized=False)
    haversine_link_lengths = []

    for edge in G.edges():
        node1, node2 = edge
        # Calculate the link length (Haversine distance between nodes)
        lat1, lon1 = G.nodes[node1]["lat"], G.nodes[node1]["long"]
        lat2, lon2 = G.nodes[node2]["lat"], G.nodes[node2]["long"]

        # Calculate link length using haversine
        haversine_link_length= haversine_distance(lat1, lon1, lat2, lon2)
        haversine_link_lengths.append( haversine_link_length)
        computed_fiber_link_length = compute_fiber_length(haversine_link_length)

        # Set edge attributes
        G.edges[edge]["edge_betweenness"] = edge_betweenness.get(edge, 0.0)
        G.edges[edge]["haversine_link_length"] = haversine_link_length
        G.edges[edge]["conputed_fiber_link_length"] =  computed_fiber_link_length

    return G, haversine_link_lengths


def visualize_cost_on_folium(G):
    # Create a base map centered around Europe
    m = folium.Map(location=[54, 15], zoom_start=4)

    # Add nodes to the map
    for node, attributes in G.nodes(data=True):
        lat = attributes['lat']
        long = attributes['long']
        folium.Marker([lat, long],
                      icon=folium.Icon(color='blue', icon='circle', prefix='fa')).add_to(m)

    # Add edges to the map
    for u, v in G.edges():
        start_lat = G.nodes[u]['lat']
        start_long = G.nodes[u]['long']
        end_lat = G.nodes[v]['lat']
        end_long = G.nodes[v]['long']
        coordinates = [[start_lat, start_long], [end_lat, end_long]]
        folium.PolyLine(coordinates, color="navy", weight=2).add_to(m)

    return m

def visualize_cost_on_folium_with_edge_length(G): # DOES NOT look pretty on the graph - don't use it
    # Create a base map centered around Europe
    m = folium.Map(location=[54, 15], zoom_start=4)

    # Add nodes to the map
    for node, attributes in G.nodes(data=True):
        lat = attributes['lat']
        long = attributes['long']
        folium.Marker([lat, long], icon=folium.Icon(color='blue', icon='circle', prefix='fa')).add_to(m)

    # Add edges to the map with edge lengths as labels
    for u, v, data in G.edges(data=True):
        start_lat = G.nodes[u]['lat']
        start_long = G.nodes[u]['long']
        end_lat = G.nodes[v]['lat']
        end_long = G.nodes[v]['long']
        coordinates = [[start_lat, start_long], [end_lat, end_long]]

        # Get the edge length from edge attributes ("haversine_link_length")
        edge_length = data.get("haversine_link_length", None)

        # Add the edge to the map with a label for edge length
        folium.PolyLine(coordinates, color="navy", weight=2).add_to(m)
        if edge_length is not None:
            label = f"Edge Length: {edge_length:.2f} km"
            folium.Marker([(start_lat + end_lat) / 2, (start_long + end_long) / 2],
                          icon=folium.DivIcon(html=f'<div>{label}</div>')).add_to(m)

    return m


def print_edge_attributes(G):
    for edge in G.edges():
      node1, node2 = edge
      edge_betweenness = G.edges[edge]["edge_betweenness"]
      haversine_link_length = G.edges[edge]["haversine_link_length"]
      computed_fiber_link_length = G.edges[edge]["conputed_fiber_link_length"]
      print(f"Edge ({node1}, {node2}): Edge Betweenness = {edge_betweenness}, Haversine Link Length = {haversine_link_length}, Computed Fiber Link Length = {computed_fiber_link_length}")
      # print(computed_fiber_link_length)



def compute_fiber_length(haversine_distance):
    """
    Compute the length of fiber based on haversine distance.

    Parameters:
    - haversine_distance (float): Haversine distance in kilometers.

    Returns:
    - float: Length of fiber in kilometers.
    """

    if haversine_distance < 1000:
        return 1.5 * haversine_distance
    elif 1000 <= haversine_distance <= 1200:
        return 1500
    else:
        return 1.25 * haversine_distance


def create_coronet_conus_60_graph():
  # Create an empty graph
    G = nx.Graph()

    # Nodes
    nodes = [i for i in range(1, 60)]
    G.add_nodes_from(nodes)

    # Edges
    # Add edges and weights
    edges = [
              (1, 8, 277.1),
              (1, 53, 234.2),
              (2, 16, 647.7),
              (2, 18, 436.9),
              (3, 7, 266.2),
              (3, 10, 439.2),
              (3, 22, 554.1),
              (4, 37, 179.2),
              (4, 59, 67.2),
              (5, 21, 500.2),
              (5, 32, 147.4),
              (6, 51, 1293.1),
              (6, 30, 1468),
              (7, 31, 352.4),
              (7, 32, 582.5),
              (8, 41, 79.9),
              (9, 53, 272),
              (9, 13, 336.4),
              (10, 19, 159.9),
              (11, 52, 501.6),
              (11, 17, 459.1),
              (11, 29, 165.3),
              (12, 14, 193.2),
              (12, 27, 177.5),
              (12, 59, 777.1),
              (13, 14, 239),
              (13, 56, 191.2),
              (14, 39, 294.7),
              (15, 58, 560),
              (15, 18, 1189.9),
              (15, 21, 432.7),
              (15, 25, 554),
              (16, 36, 920.3),
              (16, 44, 731.3),
              (17, 56, 107.2),
              (18, 45, 964.5),
              (18, 57, 505.7),
              (19, 59, 508),
              (19, 27, 690.4),
              (19, 42, 131.1),
              (20, 33, 215.6),
              (20, 41, 125.6),
              (21, 45, 425),
              (22, 42, 809),
              (22, 60, 522),
              (23, 36, 314),
              (23, 52, 470.9),
              (23, 58, 418.4),
              (24, 35, 786),
              (24, 26, 484.5),
              (24, 38, 495.9),
              (24, 44, 700),
              (25, 31, 639.2),
              (26, 46, 223.8),
              (26, 49, 150.7),
              (27, 31, 295.1),
              (27, 52, 473.8),
              (28, 55, 397.1),
              (28, 60, 129.8),
              (29, 30, 568.3),
              (30, 36, 561.3),
              (32, 54, 653.4),
              (33, 34, 24.2),
              (33, 50, 199.6),
              (34, 37, 136.1),
              (35, 43, 132.6),
              (35, 44, 1135.7),
              (35, 47, 25.7),
              (37, 50, 193.4),
              (38, 46, 574.7),
              (38, 57, 222.5),
              (39, 50, 473.6),
              (40, 43, 937.7),
              (40, 51, 279.1),
              (44, 51, 1463.4),
              (47, 48, 77.2),
              (48, 49, 447),
              (50, 53, 223.8),
              (54, 55, 394.1) ]

    # Add weighted edges to the graph
    G.add_weighted_edges_from(edges)


    # Node attributes (latitude and longitude)
    node_attributes = {
                        1: {"lat": 42.67, "long": -73.8},
                        2: {"lat": 35.12, "long": -106.62},
                        3: {"lat": 33.76, "long": -84.42},
                        4: {"lat": 39.3, "long": -76.61},
                        5: {"lat": 30.45, "long": -91.13},
                        6: {"lat": 45.79, "long": -108.54},
                        7: {"lat": 33.53, "long": -86.8},
                        8: {"lat": 42.34, "long": -71.02},
                        9: {"lat": 42.89, "long": -78.86},
                        10: {"lat": 35.2, "long": -80.83},
                       11: {"lat": 41.84, "long": -87.68},
                        12: {"lat": 39.14, "long": -84.51},
                        13: {"lat": 41.48, "long": -81.68},
                        14: {"lat": 39.99, "long": -82.99},
                        15: {"lat": 32.79, "long": -96.77},
                        16: {"lat": 39.77, "long": -104.87},
                        17: {"lat": 42.38, "long": -83.1},
                        18: {"lat": 31.85, "long": -106.44},
                        19: {"lat": 36.08, "long": -79.83},
                        20: {"lat": 41.77, "long": -72.68},
                        21: {"lat": 29.77, "long": -95.39},
                        22: {"lat": 30.33, "long": -81.66},
                        23: {"lat": 39.12, "long": -94.73},
                        24: {"lat": 36.21, "long": -115.22},
                        25: {"lat": 34.72, "long": -92.35},
                        26: {"lat": 34.11, "long": -118.41},
                        27: {"lat": 38.22, "long": -85.74},
                        28: {"lat": 25.78, "long": -80.21},
                        29: {"lat": 43.06, "long": -87.97},
                        30: {"lat": 44.96, "long": -93.27},
                        31: {"lat": 36.17, "long": -86.78},
                        32: {"lat": 30.07, "long": -89.93},
                        33: {"lat": 40.67, "long": -73.94},
                        34: {"lat": 40.72, "long": -74.17},
                        35: {"lat": 37.77, "long": -122.22},
                        36: {"lat": 41.26, "long": -96.01},
                        37: {"lat": 40.01, "long": -75.13},
                        38: {"lat": 33.54, "long": -112.07},
                        39: {"lat": 40.3, "long": -80.13},
                        40: {"lat": 45.54, "long": -122.66},
                        41: {"lat": 41.82, "long": -71.42},
                        42: {"lat": 35.82, "long": -78.66},
                        43: {"lat": 38.57, "long": -121.47},
                        44: {"lat": 40.78, "long": -111.93},
                        45: {"lat": 29.46, "long": -98.51},
                        46: {"lat": 32.81, "long": -117.14},
                        47: {"lat": 37.66, "long": -122.42},
                        48: {"lat": 37.3, "long": -121.85},
                        49: {"lat": 34.43, "long": -119.72},
                        50: {"lat": 41.4, "long": -75.67},
                        51: {"lat": 47.62, "long": -122.35},
                        52: {"lat": 38.64, "long": -90.24},
                        53: {"lat": 43.04, "long": -76.14},
                        54: {"lat": 30.46, "long": -84.28},
                        55: {"lat": 27.96, "long": -82.48},
                        56: {"lat": 41.66, "long": -83.58},
                        57: {"lat": 32.2, "long": -110.89},
                        58: {"lat": 36.13, "long": -95.92},
                        59: {"lat": 38.91, "long": -77.02},
                        60: {"lat": 26.75, "long": -80.13}
                    }

    # Setting node attributes
    nx.set_node_attributes(G, node_attributes)

    return G


def visualize_coronet_conus_60_on_folium(G):
    # Create a base map centered around the USA
    m = folium.Map(location=[39.8283, -98.5795], zoom_start=4)

    # Add nodes to the map
    for node, attributes in G.nodes(data=True):
        lat = attributes['lat']
        long = attributes['long']
        folium.Marker([lat, long],
                      icon=folium.Icon(color='blue', icon='circle', prefix='fa')).add_to(m)

    # Add edges to the map
    for u, v in G.edges():
        start_lat = G.nodes[u]['lat']
        start_long = G.nodes[u]['long']
        end_lat = G.nodes[v]['lat']
        end_long = G.nodes[v]['long']
        coordinates = [[start_lat, start_long], [end_lat, end_long]]
        folium.PolyLine(coordinates, color="navy", weight=2).add_to(m)

    return m


# Create the graph
G_cost = create_cost266_graph()
G_cost, haversine_link_lengths = calculate_edge_attributes(G_cost)
# print_edge_attributes(G_cost)

# Visualize the graph on Folium
cost_map_graph = visualize_cost_on_folium(G_cost)
cost_map_graph.save('cost266_network_map.html')

# # Create the graph
G_coronet = create_coronet_conus_60_graph()
G_coronet, haversine_link_lengths = calculate_edge_attributes(G_coronet)
# print_edge_attributes(G_coronet)

# Visualize the graph on Folium
coronet_map_graph = visualize_coronet_conus_60_on_folium(G_coronet)
coronet_map_graph.save('Coronet_conus_60_network_map.html')


In [8]:
cost_map_graph

In [9]:
coronet_map_graph

# Link predictions

In [3]:
def run_link_prediction(G, percentages, seed):
    # Set the seed for the part of the code that involves randomness
    random.seed(seed)

    for percent in percentages:
        print(f"{percent} percent links removed and predicted")
        print("----------------------------------------------------------------------")
        # Here, the function 'execute_link_prediction_methods' should use the randomness
        # (e.g., for splitting data, shuffling, or any stochastic method). It's assumed to be affected by the seed.
        execute_link_prediction_methods(G, percent)
        print("----------------------------------------------------------------------")

def run_predictions_for_seeds(G, seeds, percentages):
    for seed in seeds:
        print("----------------------------------------------------------------------")
        print("Seed:", seed)
        print("----------------------------------------------------------------------")
        run_link_prediction(G, percentages, seed)


In [4]:
# Usage:
seeds_to_test = [42]
percentages_to_test = range(50, 91, 40)

# COST
print("---------------COST--------------")
run_predictions_for_seeds(G=G_cost, seeds=seeds_to_test, percentages=percentages_to_test)


---------------COST--------------
----------------------------------------------------------------------
Seed: 42
----------------------------------------------------------------------
50 percent links removed and predicted
----------------------------------------------------------------------
Method: Jaccard Coefficient
Number of Links Removed (and Number of Links Predicted): 28
Accuracy: 0.00%

Method: Adamic Adar Index
Number of Links Removed (and Number of Links Predicted): 28
Accuracy: 3.57%

Method: Preferential Attachment
Number of Links Removed (and Number of Links Predicted): 28
Accuracy: 7.14%

Method: Resource Allocation Index
Number of Links Removed (and Number of Links Predicted): 28
Accuracy: 0.00%

Method: Common Neighbors
Number of Links Removed (and Number of Links Predicted): 28
Accuracy: 0.00%

Method: Triadic Closure
Number of Links Removed (and Number of Links Predicted): 28
Accuracy: 0.00%

Method: Random
Number of Links Removed (and Number of Links Predicted): 28

In [6]:
# CORONET
print("---------------COROENT--------------")
run_predictions_for_seeds(G=G_coronet, seeds=seeds_to_test, percentages=percentages_to_test)

---------------COROENT--------------
----------------------------------------------------------------------
Seed: 42
----------------------------------------------------------------------
50 percent links removed and predicted
----------------------------------------------------------------------
Method: Jaccard Coefficient
Number of Links Removed (and Number of Links Predicted): 39
Accuracy: 0.00%

Method: Adamic Adar Index
Number of Links Removed (and Number of Links Predicted): 39
Accuracy: 2.56%

Method: Preferential Attachment
Number of Links Removed (and Number of Links Predicted): 39
Accuracy: 0.00%

Method: Resource Allocation Index
Number of Links Removed (and Number of Links Predicted): 39
Accuracy: 2.56%

Method: Common Neighbors
Number of Links Removed (and Number of Links Predicted): 39
Accuracy: 2.56%

Method: Triadic Closure
Number of Links Removed (and Number of Links Predicted): 39
Accuracy: 2.56%

Method: Random
Number of Links Removed (and Number of Links Predicted):